# Testing coordinate-free operator expressions

## Author:
- **David W. Hogg** (NYU)

## To-do items:
- Only some of the paper's results are tested so far.

In [ ]:
import numpy as np
import pylab as plt
# plt.rcParams.update({"text.usetex": True})
rng = np.random.default_rng(17)

In [ ]:
def inner(u, v):
    return u.T @ METRIC @ v

def outer(u, v):
    return np.outer(u, v) @ METRIC

def norm(v):
    return np.sqrt(np.abs(inner(v, v)))

def cosine(u, v):
    return inner(u, v) / norm(u) / norm(v)

def bra(u):
    return u.T @ METRIC / inner(u, u)

def ket(u):
    return u

def braket(u, v):
    return bra(u) @ ket(v)

def ketbra(u, v):
    """
    ## Bugs:
    - Ugly broadcast.
    """
    return ket(u)[:, None] @ bra(v)[None, :]

In [ ]:
def orthogonalize(vs):
    dpluss, _ = METRIC.shape
    n, dps = vs.shape
    assert n <= dpluss
    assert dps == dpluss
    us = np.zeros_like(vs)
    for i in range(n):
        ui = 1. * vs[i]
        for j in range(i):
            ui -= (inner(vs[i], us[j]) / inner(us[j], us[j])) * us[j]
        us[i] = ui
    return us

def orthonormalize(vs):
    n, dee = vs.shape
    us = orthogonalize(vs)
    for i in range(n):
        us[i] /= norm(us[i])
    return us

In [ ]:
def projector_from_vectors(vs):
    uhats = orthonormalize(vs)
    return np.sum(np.array([ketbra(u, u) for u in uhats]), axis=0)

TINY = 1.e-10 # loose
def transform_from_bases(uhats, vhats):
    assert len(uhats) == len(vhats)
    assert len(uhats) == len(METRIC)
    for u, v in zip(uhats, vhats):
        assert (inner(u, u) - inner(v, v)) ** 2 < TINY
    return np.sum(np.array([ketbra(v, u) for u, v in zip(uhats, vhats)]), axis=0)

In [ ]:
def projector_from_vector_pair(u, v):
    assert inner(u, u) * inner(v, v) > 0. # both timelike or both spacelike
    uhat, vhat = u / norm(u), v / norm(v)
    return (ketbra(uhat, uhat) + ketbra(vhat, vhat)
            - braket(uhat, vhat) * (ketbra(uhat, vhat) + ketbra(vhat, uhat))) \
    / (1. - braket(uhat, vhat) ** 2)
    
def transform_from_vector_pair(u, v):
    dpluss = len(METRIC)
    Pi = projector_from_vector_pair(u, v) # this function does the assert
    Pi_perp = np.identity(dpluss) - Pi
    uhat, vhat = u / norm(u), v / norm(v)
    return ketbra(vhat, uhat) - ketbra(uhat, vhat) + braket(uhat, vhat) * Pi + Pi_perp

In [ ]:
s, d = 1, 3 # Lorentz space
foo = np.ones(d + s)
foo[s:] = -1.
METRIC = np.diag(foo)

In [ ]:
# quick test of orthonormalize
vs = 7.0 * rng.normal(size=(len(METRIC), len(METRIC)))
us = orthonormalize(vs)
for u in us:
    for uu in us:
        print(inner(u, uu))

In [ ]:
# quick test of projector
print(projector_from_vectors(vs))

In [ ]:
# now do industrial tests of everything -- this is a lot!
ntrial = 2 ** 10
Euclidean_metric = np.eye(4)
Lorentz_metric = np.diag([1., -1., -1., -1.])
for METRIC in [Euclidean_metric, Lorentz_metric]:
    dps = len(METRIC)
    if np.allclose(METRIC, Euclidean_metric):
        title = "Euclidean"
    else:
        title = "Lorentz"
    npass = 0
    for trial in range(ntrial):
        while True:
            uu, vv = rng.normal(size=dps), rng.normal(size=dps)
            if inner(uu, uu) > 0. and inner(vv, vv) > 0.: # both timelike
                break
        # Make a transform from the two vectors
        Q = transform_from_vector_pair(uu, vv)
        # Make two orthonormal bases
        vecs = rng.normal(size=METRIC.shape)
        vecs[0] = uu
        uhats = orthonormalize(vecs)
        vecs[0] = vv
        vhats = orthonormalize(vecs)
        Q2 = transform_from_bases(uhats, vhats)
        if (True
            and np.allclose(Q.T @ METRIC @ Q, METRIC)          # is it a Lorentz transform?
            and np.allclose(Q @ uu / norm(uu), vv / norm(vv))  # is it the correct Lorentz transform?
            and np.allclose(Q2.T @ METRIC @ Q2, METRIC)        # is it a Lorentz transform?
            and np.allclose(Q2 @ uu / norm(uu), vv / norm(vv)) # is it the correct Lorentz transform?
           ):
            npass += 1
    print("testing " + title + " operators: passed", npass, "/", ntrial,
          f"({100*npass/ntrial:.1f} percent)")